### Metro Nashville Emergency Communications

#### Snickers: Hypothesis: tie between economic status in NW region of Davidson and number of calls received, specifically lack of calls. Very few calls from northwest region of Davidson. Correlate income per zipcode with calls for each storm.

Snickers: North nashville, compare calls/dispatches to full county

#### Timeline of each storm/ Beginning and end of the event
Tornado calls: 3/3/2020 12:34am-4:58am

Tornado cad: 12:36am-4:57am

Derecho calls: 5/3/2020 4:30pm- 5/4/2020 12:29am

Derecho cad: 4:31pm-12:29am

In [169]:
import pandas as pd
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

### Reading in data

In [188]:
derecho_calls=pd.read_csv("../data/911_Phone Calls_Derecho_050320 050420.csv")
tornado_calls=pd.read_csv("../data/911_Phone_Calls_Tornado_030320.csv")
derecho_cad=pd.read_csv("../data/Computer Aided Dispatch Data_Derecho Incidents 050320-050420.csv", skiprows=3)
tornado_cad=pd.read_csv("../data/Computer Aided Dispatch Data_Tornado Incidents 030320.csv", skiprows=3)
sirens = pd.read_csv('../data/Tornado_Siren_Locations.csv')
dispatch = pd.read_csv('../data/dispatch.csv')
zipcodes = gpd.read_file('../data/zipcodes.geojson')
print(zipcodes.crs)

epsg:4326


### Renaming and setting up columns

In [189]:
tornado_calls.columns = ['datetime', 'calltype','lat','lng','address']
derecho_cad.columns = ['datetime', 'calltype','lat','lng','alarm']
tornado_cad.columns = ['datetime', 'calltype','lat','lng','alarm']
derecho_calls.columns = ['datetime', 'calltype','lat','lng','address']
zipcodes = zipcodes[['zip', 'po_name', 'geometry']]
dispatch=dispatch[[ 'tencode', 'lat', 'lng', 'call_date','call_time']]

In [190]:
sirens[["lat", "lng"]]= sirens.pop("Location").str.split(", ", n = 1, expand = True)
sirens['lat']= sirens['lat'].str[1:]
sirens['lng'] = sirens['lng'].str[:-1]
sirens=sirens[["Site Name", "Zip","lat","lng"]]
sirens.head(2)

,Site Name,Zip,lat,lng
0,Ezell Road Park,37211,36.090944,-86.686028
1,Sevier Park,37204,36.118861,-86.787889


### Creating geometry column for dfs

In [191]:
derecho_calls['geometry'] = derecho_calls.apply(lambda x: Point((float(x.lng), 
                                                         float(x.lat))), 
                                        axis=1)
tornado_calls['geometry'] = tornado_calls.apply(lambda x: Point((float(x.lng), 
                                                         float(x.lat))), 
                                        axis=1)
tornado_cad['geometry'] = tornado_cad.apply(lambda x: Point((float(x.lng), 
                                                         float(x.lat))), 
                                        axis=1)
derecho_cad['geometry'] = derecho_cad.apply(lambda x: Point((float(x.lng), 
                                                         float(x.lat))), 
                                        axis=1)
dispatch['geometry'] = dispatch.apply(lambda x: Point((float(x.lng), 
                                                         float(x.lat))), 
                                        axis=1)
sirens['geometry'] = sirens.apply(lambda x: Point((float(x.lng), 
                                                         float(x.lat))), 
                                        axis=1)

In [192]:
zipcodes.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [193]:
tornado_calls_geo = gpd.GeoDataFrame(tornado_calls, 
                           crs = zipcodes.crs, 
                           geometry = tornado_calls['geometry'])
type(tornado_calls_geo)
derecho_calls_geo = gpd.GeoDataFrame(derecho_calls, 
                           crs = zipcodes.crs, 
                           geometry = derecho_calls['geometry'])
type(derecho_calls_geo)
tornado_cad_geo = gpd.GeoDataFrame(tornado_cad, 
                           crs = zipcodes.crs, 
                           geometry = tornado_cad['geometry'])
type(tornado_cad_geo)
derecho_cad_geo = gpd.GeoDataFrame(derecho_cad, 
                           crs = zipcodes.crs, 
                           geometry = derecho_cad['geometry'])
type(derecho_cad_geo)
dispatch_geo = gpd.GeoDataFrame(dispatch, 
                           crs = zipcodes.crs, 
                           geometry = dispatch['geometry'])
type(dispatch_geo)
sirens_geo = gpd.GeoDataFrame(sirens, 
                           crs = zipcodes.crs, 
                           geometry = dispatch['geometry'])
type(sirens_geo)

geopandas.geodataframe.GeoDataFrame

### Joining dfs

In [196]:
tornado_calls_zip = gpd.sjoin(tornado_calls_geo, zipcodes, op = 'within')
tornado_calls_zip=tornado_calls_zip.reset_index(drop = True)
tornado_cad_zip = gpd.sjoin(tornado_cad_geo, zipcodes, op = 'within')
tornado_cad_zip=tornado_cad_zip.reset_index(drop = True)
derecho_calls_zip = gpd.sjoin(derecho_calls_geo, zipcodes, op = 'within')
derecho_calls_zip=derecho_calls_zip.reset_index(drop = True)
derecho_cad_zip = gpd.sjoin(derecho_cad_geo, zipcodes, op = 'within')
derecho_cad_zip=derecho_cad_zip.reset_index(drop = True)
dispatch_zip = gpd.sjoin(dispatch_geo, zipcodes, op = 'within')
dispatch_zip=dispatch_zip.reset_index(drop = True)

### Drilling down to NW Nash

In [197]:
poly_zip=zipcodes[zipcodes['zip'].isin(['37207', '37072', '37189', '37218','37015','37080'])]
poly_zip=poly_zip.reset_index(drop = True)
tcalls_in_nw = tornado_calls_zip[tornado_calls_zip['zip'].isin(['37207', '37072', '37189', '37218','37015','37080'])]
tcalls_in_nw=tcalls_in_nw.reset_index(drop = True)
tcad_in_nw = tornado_cad_zip[tornado_cad_zip['zip'].isin(['37207', '37072', '37189', '37218','37015','37080'])]
tcad_in_nw=tcad_in_nw.reset_index(drop = True)
dcalls_in_nw = derecho_calls_zip[derecho_calls_zip['zip'].isin(['37207', '37072', '37189', '37218','37015','37080'])]
dcalls_in_nw=dcalls_in_nw.reset_index(drop = True)
dcad_in_nw = derecho_cad_zip[derecho_cad_zip['zip'].isin(['37207', '37072', '37189', '37218','37015','37080'])]
dcad_in_nw=dcad_in_nw.reset_index(drop = True)
sirens_nw=sirens[sirens['Zip'].isin(['37207', '37072', '37189', '37218','37015','37080'])]
dispatch_nw=dispatch_zip[dispatch_zip['zip'].isin(['37207', '37072', '37189', '37218','37015','37080'])]

### Merging all df

In [26]:
final_df= tornado_calls_zip.merge(tornado_cad_zip,on='zip').merge(derecho_calls_zip,on='zip').merge(derecho_cad_zip,on='zip').merge(dispatch_zip,on='zip')


KeyboardInterrupt: 

### Separating storms from dispatch data

In [ ]:
dispatch_tornado=dispatch_zip[dispatch_zip['call_date'].isin(['03/03/2020'])]
dispatch_tornado=dispatch_tornado.reset_index(drop = True)
dispatch_derecho=dispatch_zip[dispatch_zip['call_date'].isin(['05/03/2020','05,04,2020'])]
dispatch_derecho=dispatch_derecho.reset_index(drop = True)

In [ ]:
nashville = (36.16784, -86.77816)

nash_map = folium.Map(location = nashville, zoom_start = 10)

In [ ]:
nash_map = folium.Map(location = nashville, zoom_start = 10)
image_file = '../data/map_legend.png'
FloatImage(image_file, bottom=5, left=70).add_to(nash_map)
feature_group = folium.FeatureGroup(name='Districts', show=True )
feature_group1 = folium.FeatureGroup(name='School District 1', show=False )
feature_group2 = folium.FeatureGroup(name='School District 2', show=False)
feature_group3 = folium.FeatureGroup(name='School District 3', show=False)
feature_group4 = folium.FeatureGroup(name='School District 4', show=False)
feature_group5 = folium.FeatureGroup(name='School District 5', show=False)
feature_group6 = folium.FeatureGroup(name='School District 6', show=False)
feature_group7 = folium.FeatureGroup(name='School District 7', show=False)
feature_group8 = folium.FeatureGroup(name='School District 8', show=False)
feature_group9 = folium.FeatureGroup(name='School District 9', show=False)
nash_map.add_child(feature_group)
nash_map.add_child(feature_group1)
nash_map.add_child(feature_group2)
nash_map.add_child(feature_group3)
nash_map.add_child(feature_group4)
nash_map.add_child(feature_group5)
nash_map.add_child(feature_group6)
nash_map.add_child(feature_group7)
nash_map.add_child(feature_group8)
nash_map.add_child(feature_group9)
folium.GeoJson(school_districts['geometry']).add_to(feature_group)
marker_cluster1 = MarkerCluster().add_to(feature_group1)
marker_cluster2 = MarkerCluster().add_to(feature_group2)
marker_cluster3 = MarkerCluster().add_to(feature_group3)
marker_cluster4 = MarkerCluster().add_to(feature_group4)
marker_cluster5 = MarkerCluster().add_to(feature_group5)
marker_cluster6 = MarkerCluster().add_to(feature_group6)
marker_cluster7 = MarkerCluster().add_to(feature_group7)
marker_cluster8 = MarkerCluster().add_to(feature_group8)
marker_cluster9 = MarkerCluster().add_to(feature_group9)
def color(Students):
    if Students <= 100.0:
        col = 'green'
    elif Students <= 200.0:
        col = 'blue'
    elif Students <= 300.0:
        col = 'yellow'
    else:
        col='red'
    return col
for row_index, row_values in nct_19_d1.iterrows():
    loc = [row_values['lat'], row_values['long']]
    pop = str(row_values['School Name']) + '<br><br> Category: ' + str(row_values['Category']) + '<br><br> Students: ' + str(row_values['Students'])
    icon=folium.Icon(color=color(row_values['Students']), icon = 'building', prefix='fa', textalign = 'center')
    marker = folium.Marker(tooltip = 'School District 1',
        location = loc,
        popup = pop, icon = icon).add_to(feature_group1)
    marker.add_to(marker_cluster1)
for row_index, row_values in nct_19_d2.iterrows():
    loc = [row_values['lat'], row_values['long']]
    pop = str(row_values['School Name']) + '<br><br> Category: ' + str(row_values['Category']) +'<br><br> Students: ' + str(row_values['Students'])
    icon=folium.Icon(color=color(row_values['Students']), icon = 'building', prefix='fa', textalign = 'center')
    marker = folium.Marker(tooltip = 'School District 2',
        location = loc,
        popup = pop, icon = icon).add_to(feature_group2)
    marker.add_to(marker_cluster2)
for row_index, row_values in nct_19_d3.iterrows():
    loc = [row_values['lat'], row_values['long']]
    pop = str(row_values['School Name']) + '<br><br> Category: ' + str(row_values['Category']) + '<br><br> Students: ' + str(row_values['Students'])
    icon=folium.Icon(color=color(row_values['Students']), icon = 'building', prefix='fa', textalign = 'center')
    marker = folium.Marker(tooltip = 'School District 3',
        location = loc,
        popup = pop, icon = icon).add_to(feature_group3)
    marker.add_to(marker_cluster3)
for row_index, row_values in nct_19_d4.iterrows():
    loc = [row_values['lat'], row_values['long']]
    pop = str(row_values['School Name']) + '<br><br> Category: ' + str(row_values['Category']) + '<br><br> Students: ' + str(row_values['Students'])
    icon=folium.Icon(color=color(row_values['Students']), icon = 'building', prefix='fa', textalign = 'center')
    marker = folium.Marker(tooltip = 'School District 4',
        location = loc,
        popup = pop, icon = icon).add_to(feature_group4)
    marker.add_to(marker_cluster4)
for row_index, row_values in nct_19_d5.iterrows():
    loc = [row_values['lat'], row_values['long']]
    pop = str(row_values['School Name']) + '<br><br> Category: ' + str(row_values['Category']) + '<br><br> Students: ' + str(row_values['Students'])
    icon=folium.Icon(color=color(row_values['Students']), icon = 'building', prefix='fa', textalign = 'center')
    marker = folium.Marker(tooltip = 'School District 5',
        location = loc,
        popup = pop, icon = icon).add_to(feature_group5)
    marker.add_to(marker_cluster5)
for row_index, row_values in nct_19_d6.iterrows():
    loc = [row_values['lat'], row_values['long']]
    pop = str(row_values['School Name']) + '<br><br> Category: ' + str(row_values['Category']) + '<br><br> Students: ' + str(row_values['Students'])
    icon=folium.Icon(color=color(row_values['Students']), icon = 'building', prefix='fa', textalign = 'center')
    marker = folium.Marker(tooltip = 'School District 6',
        location = loc,
        popup = pop, icon = icon).add_to(feature_group6)
    marker.add_to(marker_cluster6)
for row_index, row_values in nct_19_d7.iterrows():
    loc = [row_values['lat'], row_values['long']]
    pop = str(row_values['School Name']) + '<br><br> Category: ' + str(row_values['Category']) + '<br><br> Students: ' + str(row_values['Students'])
    icon=folium.Icon(color=color(row_values['Students']), icon = 'building', prefix='fa', textalign = 'center')
    marker = folium.Marker(tooltip = 'School District 7',
        location = loc,
        popup = pop, icon = icon).add_to(feature_group7)
    marker.add_to(marker_cluster7)
for row_index, row_values in nct_19_d8.iterrows():
    loc = [row_values['lat'], row_values['long']]
    pop = str(row_values['School Name']) + '<br><br> Category: ' + str(row_values['Category']) + '<br><br> Students: ' + str(row_values['Students'])
    icon=folium.Icon(color=color(row_values['Students']), icon = 'building', prefix='fa', textalign = 'center')
    marker = folium.Marker(tooltip = 'School District 8',
        location = loc,
        popup = pop, icon = icon).add_to(feature_group8)
    marker.add_to(marker_cluster8)
for row_index, row_values in nct_19_d9.iterrows():
    loc = [row_values['lat'], row_values['long']]
    pop = str(row_values['School Name']) + '<br><br> Category: ' + str(row_values['Category']) + '<br><br> Students: ' + str(row_values['Students'])
    icon=folium.Icon(color=color(row_values['Students']), icon = 'building', prefix='fa', textalign = 'center')
    marker = folium.Marker(tooltip = 'School District 9',
        location = loc,
        popup = pop, icon = icon).add_to(feature_group9)
    marker.add_to(marker_cluster9)
theatre_location = folium.Marker(location = [36.154550, -86.767910], popup = "Nashville Children's Theatre" + '<br>FY19', icon = folium.Icon(color = 'purple', icon = 'ticket', prefix = 'fa'))
theatre_location.add_to(nash_map)
turn on layer control
nash_map.add_child(folium.map.LayerControl(collapsed = False))
nash_map.save('../maps/nct_grants_map.html')

In [ ]:
new_map_dcad = folium.Map(location =  nashville, zoom_start = 14)

marker_cluster = MarkerCluster().add_to(new_map_dcad)

folium.GeoJson(zipcodes).add_to(marker_cluster)

for row_index, row_values in derecho_cad_zip.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['alarm'])
    icon=folium.Icon(color="red",icon="phone", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(marker_cluster)

    
#new_map_dcad.save('../maps/derecho_cad_nash.html')

#new_map_dcad 

In [ ]:
leg_kwds = {'title': 'Zipcodes', 'loc': 'upper left', 
            'bbox_to_anchor': (1, 1.03), 'ncol': 2}

zipcodes.plot(column = 'zip', figsize=(10, 10),
              edgecolor = 'black',
              legend = True, legend_kwds = leg_kwds, 
              cmap = 'tab20c')
plt.show()

In [ ]:
map_zipcodes = folium.Map(location =  nashville, zoom_start = 12)

folium.GeoJson(zipcodes).add_to(map_zipcodes)

for row_index, row_values in tornado_calls_zip.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['zip'])
    icon=folium.Icon(color="red",icon="phone", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop, icon = icon) 

    marker.add_to(map_zipcodes)
    
#map_zipcodes.save('../maps/mapzipcodes.html')

#map_zipcodes

In [ ]:
cluster_map_nw = folium.Map(location =  nashville, zoom_start = 12)

marker_cluster = MarkerCluster().add_to(cluster_map_nw)

folium.GeoJson(poly_zip).add_to(cluster_map_nw)

for row_index, row_values in dcalls_in_nw.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['address'])
    icon=folium.Icon(color="red",icon="phone", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(marker_cluster)

    
cluster_map_nw.save('../maps/clusternw.html')

cluster_map_nw  